In [26]:
import biosteam as bst
import thermosteam as tmo
import biorefineries as bf
from biorefineries import cellulosic
from lignin_saf.reactor import *

In [27]:


def create_chemicals():


    # copying cellulosic ethanol chemicals 
    chems = cellulosic.create_cellulosic_ethanol_chemicals().copy()

    # Adding pre-existing chemicals
    methanol = tmo.Chemical('Methanol')
    hydrogen = tmo.Chemical('Hydrogen')

    
    # Custom chemical properties estimated using NIST ThermoDataEngine (TDE) from Aspen Plus V14
    # Only propylguaiacol was native to Aspen, every other component was user defined
   
    propylguaiacol = tmo.Chemical(  # S-Lignin Monomer 1
        'Propylguaiacol',
        default = True,      # Defaults all other properties 
        search_db=False,     # Since not present in database, do not search
        formula='C10H14O2',  # Chemical formulae
        phase='l',           # phase at rtp
        omega = 0.6411,      # accentric factor
        Tb = 541.7,          # [K]  normal boiling point
        Tc = 749,            # [K]  critical temperature
        Pc = 2.9e6,          # [Pa] critical pressure
        Hvap = 7.78e4,       # [J/mol] enthalpy of vaporization at 298 K
        rho = 1056.3,        # [kg/m3] density at rtp

    )
    propylguaiacol.synonyms = ('4-Propylguaiacol',) # Synonyms that can be used to refer it as

    
    propylsyringol = tmo.Chemical(   # G-Lignin Monomer 1
        'Propylsyringol',
        default = True,
        search_db=False,
        formula='C11H16O3',
        phase='l',
        omega = 0.87334, 
        Tb = 617.3,      
        Tc = 819,        
        Pc = 2565348.3,   
        Hvap = 1.07e5,    
        rho  = 1274.3    
    )
    propylsyringol.synonyms = ('4-Propylsyringol',)

    syringaresinol = tmo.Chemical(  # S-Lignin Dimer 
        'Syringaresinol',
        default = True,
        search_db = False,
        formula = 'C22H26O8',
        phase = 'l',
        omega = 1.4608,
        Tb = 796.7,
        Tc = 979,
        Pc = 1.83e6,
        Hvap = 1.99e5,
        rho = 1596.5
    )

    g_dimer = tmo.Chemical(   # Couldn't find a chemical name so going with g_dimer
        'G_Dimer',
        default = True,
        search_db = False,
        MW = 362.42,
        phase = 'l',
        omega = 1.3966,
        Tb = 812.6,
        Tc = 992,
        Pc = 1.54e6,
        Hvap = 1.97e5,
        rho = 1497.3
    )

    s_oligomer = tmo.Chemical(
        'S_Oligomer',
        default = True,
        search_db = False,
        MW = 628.67,
        phase = 'l',
        omega = 0.78742,
        Tb = 921.4,
        Tc = 1128,
        Pc = 8.27e5
    )  

    g_oligomer = tmo.Chemical(   
        'G_Oligomer',
        default = True,
        search_db = False,
        MW = 540.65,
        phase = 'l'
    )


    

    # 4) Extend the base collection
    chems.extend([methanol, hydrogen, propylguaiacol, propylsyringol,syringaresinol,g_dimer,
                  s_oligomer, g_oligomer])

    chems.compile()  # Compiling all the chemicals to one string
    return chems


chems = create_chemicals()
bst.settings.set_thermo(chems) # Setting thermodynamic property pacakge for the chemicals


In [28]:
bst.settings.CEPCI = 541.7   # CEPCI for 2016, set to same as Bartling paper

In [29]:
# Defining poplar group based off composition given in Bartling et al Table S1

chems.define_group(
    name='Poplar',
    IDs=['Cellulose',   # Cellulase break this down to glucose
         'Xylan',       # Hemicellulose
         'Arabinan',    # Hemicellulose
         'Mannan',      # Hemicellulose
         'Galactan',    # Hemicellulose
         'Sucrose',   
         'Lignin',
         'Acetate',
         'Extract',     # Non structural component of biomass that dissolves in solvent/water 
         'Ash'],
    composition=[0.464,      # Dry wt composition, feed also has 20 wt% moisture content
                 0.134,
                 0.002,
                 0.037,
                 0.014,
                 0.001,
                 0.285,
                 0.035,
                 0.016,
                 0.012],
    wt=True
)


In [30]:
# Process conditions from Bartling et al 2021 unless specified otherwise


# Feed
biomass_flow = 2000                   # [dry metric tons per day], consistent with other TEA models 
biomass_moisture  = 0.2               # [%] 20% moisture

# RCF
rcf_temp = 200 + 273.15            # [K]
rcf_pressure = 60 * 1e5            # [bar]
rcf_tau = 3                        # [hr] residence time
meoh_h2o = 9                       # [ratio] Solvent : Water ratio
delignification = 0.7              # [%] 70% biomass delignified
methanol_decomp = 0.005            # [%] methanol decomposition rate
cellulose_retention = 0.9          # [%] 90% cellulose retained in biomass pulp after RCF
xylose_retention = 0.93            # [%] 93% xylose retained in biomass pulp after RCF
extractives_retention = 0          # [%] extractives retention in biomass from 10.1039/d1gc01591e Table S1 (no extractives in post-solvolysis poplar)
                                   # Also validated from SI Table S17 of Bartling et al where extractives total amount (kg) in RCF oil is similar to what is in the poplar feed 
acetate_retention = 0              # [%] acetate retention in biomass from 10.1039/d1gc01591e Table S1 (no acetate in post-solvolysis poplar)
cat_replace = 1                    # [/yr] catalyst replacement rate
methanol_to_biomass = 90           # [L/kg] from https://doi.org/10.1016/j.copbio.2018.12.005
                                   # Surprisingly, Bartling et al assumes a 9L/kg for a flow through
                                   # configuration which is very less 
hydrogen_amount = 10               # 10 L/min/dry kg biomass
catalyst_loading = 0.1             # 1:10 catalyst: dry biomass feed by wt 


# RCF oil composition
lignin_monomers = 0.5
lignin_dimers = 0.25
lignin_oligomers = 0.25 





In [31]:
poplar_in = bst.MultiStream('Poplar_In',
                s=[('Poplar', biomass_flow*1e3)], l=[('Water', biomass_moisture*biomass_flow*1e3)], 
                 phases = ('s','l'), units='kg/d')

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\_multi_stream.py:251: RuntimeWarning: <MultiStream: Poplar_In> has been replaced in registry
  self._register(ID)


In [32]:
meoh_in = bst.MultiStream('Methanol_In',
                          l = [('Methanol', methanol_to_biomass*biomass_flow*1e3*0.9),
                               ('Water', methanol_to_biomass*biomass_flow*1e3*(1/meoh_h2o)*0.9)],
                     phases = ('s','l','g'),   units = 'L/d')

# Set as a MultiStream to make it easier later on as this stream will also contain carbohydrate
# derivatives (solids) following solvolysis and hydrogen (gas) following hydrogenolysis

# both flows multipled by 0.9 to make the overall meoh_in to biomass liter to kg ratio as 90 which is a requirement

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\thermosteam\_multi_stream.py:251: RuntimeWarning: <MultiStream: Methanol_In> has been replaced in registry
  self._register(ID)


In [33]:
# Checks to ensure the correct methanol to biomass and methanol to water ratio is used
meoh_biomass_check = round((meoh_in.F_vol*1000)/(2e6/24),0)
methanol_water_check = round(meoh_in.ivol['Methanol']/meoh_in.ivol['Water'],0)

if meoh_biomass_check == methanol_to_biomass:
    print('Correct methanol to biomass ratio')

else:
    raise ValueError(f'Methanol to biomass ratio is {meoh_biomass_check} L/kg dry biomass \n It must be {methanol_to_biomass} L/kg dry biomass as specified')

if methanol_water_check == meoh_h2o:
    print('Correct methanol water ratio')

else:
    raise ValueError(f'Methanol water ratio is {methanol_water_check} \n It must be {meoh_h2o}')


Correct methanol to biomass ratio
Correct methanol water ratio


In [34]:
pump_try = bst.units.Pump(ins = meoh_in, P = 6e6)
heat_try = bst.units.HXutility(ins = pump_try-0, T = 463.15, rigorous = True)
pump_try.simulate()
heat_try.simulate()

In [35]:
heat_try.outs[0]

MultiStream: s4 from <HXutility: H2>
phases: ('g', 'l'), T: 463.15 K, P: 6e+06 Pa
flow (kmol/hr): (l) Water     4.15e+04
                    Methanol  1.66e+05


In [36]:
heat_try.outs[0].rho

605.5369549961846

In [37]:

# Solvolysis reaction to solubilize lignin from cell wall
solvolysis = bst.Reaction('Lignin,s -> SolubleLignin,l', reactant = 'Lignin', phases = 'sl', X = 0.7, basis = 'wt', correct_atomic_balance = False) 
# correct atomic balance set to False because we are working with pseudocomponents

In [92]:
"""
References
-----------
    [1] Bartling, Andrew W., et al. 
    "Techno-economic analysis and life cycle assessment of a biorefinery utilizing
    reductive catalytic fractionation." Energy & Environmental Science 14.8 (2021): 4147-4168.

    [2] Anderson, Eric M., et al. 
    "Flowthrough reductive catalytic fractionation of biomass." Joule 1.3 (2017): 613-622.

    [3] Froment, Gilbert F., Kenneth B. Bischoff, and Juray De Wilde. 
    Chemical reactor analysis and design. Vol. 2. New York: Wiley, 1990.
-----------
"""

import biosteam as bst, numpy as np

from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, 
)
 


__all__ = (
    'SolvolysisReactor', 'RCF1'
    )

class SolvolysisReactor(bst.Unit, bst.units.design_tools.PressureVessel):

    """
    Plug flow reactor for solvolysis reaction, where a solvent is used to solubilize lignin present
    in plant cell wall
    

    In reality, lignin will also be partially depolymerized, but this phenomena is not captured in 
    current design but may be included in a future implementation
    Design based off [1],[2]. Pressure drop calculations from [3]

    By default,  current design supports 4 identical biomass beds, with 3 operational beds at any given time, 
    offset by 1 hour each. With a 3 hr reaction time + 1 hr turnaround, this gives near continuous throughput
    Example... t = 0 - 1 hr : bed 1, bed 2, bed 3 online, bed 4 cleaning, 
                t = 1 - 2 hr, bed 2, bed 3, bed 4 online, bed 1 cleaning
    Since at any given time, the complete throughput is mantained by 3 reactors, the total volumetric flow rate 
    of solvent is constant

    Parameters.....


    Examples
    


    Assumed: Extraction efficiency constant along the reaction residence time


    """



 

    auxiliary_unit_names = (
        'pump_1', 'heat_exchanger_1'
    )

    _N_ins = 2
    _N_outs = 2
    
    _units = {**PressureVessel._units,
              'Pressure drop': 'bar',
              'Batch time': 'hr',
              'Turnaround time': 'hr',
              'Residence time': 'hr',
              'Total volume': 'm3',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 6 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 2

    #: Default cleaning and unloading time (hr).
    tau_0_default: float  = 1 # from https://doi.org/10.1039/D1EE01642C
    
    # Default superficial velocity of solvent (m/s)
    superficial_velocity_default: float = 1 # Just assumed 

    # Default methanol decomposition (%)
    methanol_decomposition_default: float = 0.005 # from https://doi.org/10.1039/D1EE01642C

    # Default poplar bed void fraction (epsilon)
    void_frac_default: float = 0.3    


    # Default working volume fraction 
    working_vol_default: float = 0.8
 
    # Default poplar diameter [m]
    poplar_diameter_default: float = 0.004   
    



    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            tau_0: Optional[float] = None,  
            superficial_velocity: Optional[float] = None,
            methanol_decomposition: Optional[float] = None,
            void_frac: Optional[float] = None,
            working_vol: Optional[float] = None,
            poplar_diameter: Optional[float] = None,
            N_beds = 8, *,
            reaction
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.tau_0 = self.tau_0_default if tau_0 is None else tau_0
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.methanol_decomposition = self.methanol_decomposition_default if methanol_decomposition is None else methanol_decomposition
        self.void_frac = self.void_frac_default if void_frac is None else void_frac
        self.working_vol = self.working_vol_default if working_vol is None else working_vol
        self.poplar_diameter = self.poplar_diameter_default if poplar_diameter is None else poplar_diameter
        if N_beds != 8:
            raise ValueError('only 8 biomass beds are valid')
        self.reaction = reaction
        self.N_beds = N_beds
        pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_1 = self.auxiliary('heat_exchanger_1', bst.HXutility, ins = pump_1.outs[0])


    def _size_bed(self):

        #### Reactor volume sizing ########

        cycle_time = self.tau + self.tau_0    # Total time for a cycle of operation (includes reaction + cleaning-loading-unloading)
        occupied_frac = 1 - self.void_frac    # ratio of occupied biomass fraction/total biomass bed
        working_vol = self.working_vol        # Working volume fraction for a reactor to allow suficient mass transfer by allowing enough contact time 
        solvent = self.ins[1]


        
        liquid_V_w = solvent.F_vol * cycle_time   # [m3] Total liquid working volume
        liquid_V_w_pr = liquid_V_w/self.N_beds    # [m3] Liquid working volume per reactor 
        V_actual = liquid_V_w_pr/(self.void_frac * working_vol)  # [m3]    void frac = vol. of solvent/total bed vol.
        total_volume = V_actual * self.N_beds

        ##### Reactor diameter and length ########
        u  = self.superficial_velocity                   # [m/s]

        N_operating = self.N_beds *(self.tau/cycle_time) # Number of beds operating at any given time
        Q_per_reactor_m3 = solvent.F_vol/N_operating     # [m3/hr] Volumetric flow rate processed by any reactor
        self.area = A  = Q_per_reactor_m3/(u*3600)
        self.diameter = diameter = 2 * (A/np.pi) ** 0.5
        self.length = length = V_actual/A

        
        return length, diameter, V_actual, total_volume        

        
    def _run(self):
        biomass, solvent = self.ins
        used_biomass, used_solvent = self.outs
        used_solvent.copy_like(solvent) 
        used_solvent.imass['l', 'Methanol'] = used_solvent.imass['l', 'Methanol']*(1-self.methanol_decomposition) # 0.5% methanol lost due to decomposition
        used_biomass.copy_like(biomass) 
        used_solvent.P = self.P # Outlet pressure is set to reactor pressure. Inlet pressure will be greater to account for pressure drop
        used_solvent.T = self.T # Since isothermal operation

        self.reaction(used_biomass) 
        

        solubilized_lignin = used_biomass.imass['SolubleLignin'] 
        used_solvent.imass['l', 'SolubleLignin'] += solubilized_lignin # Soluble lignin dissolves in solvent effluent stream 
        used_biomass.imass['l', 'SolubleLignin'] = 0 # No soluble lignin remaining in biomass (assuming 100% extraction efficiency)



        extractives = used_biomass.imass['Extract'] # From Table S1 https://www.rsc.org/suppdata/d1/gc/d1gc01591e/d1gc01591e1.pdf,
                                                        # it follows that the extractives component of poplar is 'extracted' in the solvent stream
        used_solvent.imass['s', 'Extract'] = (1-extractives_retention)*extractives
        used_biomass.imass['s', 'Extract'] = (extractives_retention)*extractives

        acetate = used_biomass.imass['s', 'Acetate']
        used_solvent.imass['s', 'Acetate'] =  acetate *(1-acetate_retention)
        used_biomass.imass['s', 'Acetate'] = acetate*acetate_retention


        cellulose_mass = used_biomass.imass['s', 'Cellulose']
        used_biomass.imass['s', 'Cellulose'] =  cellulose_mass*cellulose_retention
        used_solvent.imass['s', 'Cellulose'] = cellulose_mass*(1-cellulose_retention)
                                                               

        xylose_mass = used_biomass.imass['s', 'Xylose']
        used_biomass.imass['s', 'Xylose'] = xylose_mass * xylose_retention
        used_solvent.imass['s', 'Xylose'] = xylose_mass * (1-xylose_retention)
        





    def _calculate_pressure_drop(self, bed_length):

        D = self.poplar_diameter # [m] poplar particle diameter
        rho = self.outs[1].rho  # [kg/m3] 
        mu = self.outs[1].get_property('mu', 'kg/m/s') # [Pa s] methanol water viscosity 
        epsilon = self.void_frac # Void fraction 
        u = self.superficial_velocity # [m/s] superficial velocity  

        
        
        Re = (D*rho*u)/mu
        if Re/(1-epsilon) < 500: # Erun equation
            f = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
            dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5             # [bar] 1e-5 converts Pa to bar
        elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
            f = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
            dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5
        else: # Hicks equation which fits in Wentz and Thodos results for very high Re
            f = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
            dP = (f * ((rho*(u**2))/D) * bed_length)*1e-5
        return dP

        

    def _design(self):
        length, diameter, V_actual, total_volume = self._size_bed()   # Calling size bed function to determine diameter and length 
        

        cycle_time = self.tau + self.tau_0

        self.set_design_result('Diameter', 'ft', diameter)  
        self.set_design_result('Length', 'ft', length)
        self.set_design_result('Reactor volume', 'm3', V_actual)
        self.set_design_result('Total volume', 'm3', total_volume)
        self.set_design_result('Residence time', 'hr', self.tau)
        self.set_design_result('Turnaround time', 'hr', self.tau_0)
        self.set_design_result('Batch time', 'hr', cycle_time)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter']*3.28084,
                self.design_results['Length']*3.28084
            )
        )
        
                            

        pressure_drop = self._calculate_pressure_drop(length)                  
        
        self.set_design_result('Pressure drop', 'bar', pressure_drop)
        self.pump_1.P = (self.P - self.ins[1].P) + (pressure_drop*1e5)
        self.pump_1.simulate()

        self.heat_exchanger_1.T = self.T
        self.heat_exchanger_1.simulate()
        





        """

    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )

        self.parallel['self'] = self.N_beds # Used to create multiple of the same beds
        # self.parallel['pump_1'] = 1


        
       

        ---------

        - Duty requirements are too high - need to check thermodynamics of the solvolysis step
        and modify requirements if needed
          
        - Might need to alter data on working volume, void fraction of poplar, poplar particle size,
        for more realistic modeling

        ----------

        """



        

        


In [39]:
D = 0.004                   # [m] poplar particle diameter
rho = 600                   # [kg/m3] 
mu = 9.71e-05               # [Pa s] methanol water viscosity 
epsilon = 0.3               # void fraction 
u = 0.003                     # [m/s] superficial velocity  

bed_length = 150

Re = (D*rho*u)/mu
if Re/(1-epsilon) < 500: # Erun equation
    f = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
    dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5
elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
    f = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
    dP = (f * ((rho*(u**2))/D)* bed_length)*1e-5
else: # Hicks equation which fits in Wentz and Thodos results for very high Re
    f = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
    dP = (f * ((rho*(u**2))/D) * bed_length)*1e-5



In [40]:
dP

0.16621718750000006

In [41]:
meoh_in.F_vol

7500.000000000001

In [42]:
N_beds = 4
tau = 2
cycle_time = 3
N_operating  = 3
u = 0.008
void_frac = 0.4
working_vol = 0.8

total_sol_V = meoh_in.F_vol * cycle_time         # [m3]
total_sol_V_per_reactor = total_sol_V/N_beds   # [m3]
actual_V_per_reactor = total_sol_V_per_reactor/(void_frac*working_vol)

N_operating = N_beds *(tau/cycle_time) # Number of beds operating at any given time
Q_per_reactor_m3 = meoh_in.F_vol/N_operating     # [m3/hr] Volumetric flow rate processed by any reactor
A  = Q_per_reactor_m3/(u*3600)                   # [m2]
diameter = 2 * (A/np.pi) ** 0.5                  # [m]
length = actual_V_per_reactor/A                  # [m]

In [43]:
meoh_in.F_vol

7500.000000000001

In [44]:
round(100.41,1)

100.4

In [45]:
print(f'Actual reactor volume is: {actual_V_per_reactor} m3')

Actual reactor volume is: 17578.125 m3


In [46]:
diameter

11.150775725954821

In [47]:
length

179.99999999999997

In [48]:
chems['Methanol'].rho('l', 473.15, 6e6)

580.1664650372471

In [49]:
chems['Extract'].rho( 300, 101325)

100000.00472940205

In [96]:
solvolysis_reactor = SolvolysisReactor(ins = (poplar_in, meoh_in),
                                    T = rcf_temp,
                                    P = rcf_pressure,
                                    tau = 2, #rcf_tau,
                                    N_beds = 8,
                                    void_frac= 0.4,
                                    superficial_velocity = 0.005, 
                                    reaction = solvolysis)
solvolysis_reactor.simulate()

c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Poplar_In from R12; Poplar_In is now docked at R13
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\IPython\core\interactiveshell.py:3579: RuntimeWarning: undocked inlet Methanol_In from R12; Methanol_In is now docked at R13
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\hwadg\AppData\Local\Temp\ipykernel_11512\2336310535.py:264: CostWarning: <SolvolysisReactor: R13> Vertical vessel weight (2.628e+07 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_11512\2336310535.py:264: CostWarning: <SolvolysisReactor: R13> Vertical vessel length (369.1 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(


In [97]:
solvolysis_reactor

SolvolysisReactor: R13
ins...
[0] Poplar_In  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Extract    7.4
                        Acetate    48.6
                        Ash        1e+03
                        Lignin     156
                        Cellulose  238
                        Xylan      84.5
                        ...        27.5
[1] Methanol_In  
    phases: ('g', 'l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water     4.15e+04
                        Methanol  1.66e+05
outs...
[0] s28  
    phases: ('l', 's'), T: 298.15 K, P: 101325 Pa
    flow (kmol/hr): (l) Water      925
                    (s) Sucrose    0.243
                        Ash        1e+03
                        Lignin     46.8
                        Cellulose  215
                        Xylan      84.5
                        Arabinan   1.26
                        Mannan     19
 

In [98]:
solvolysis_reactor.results()


Solvolysis reactor                                      Units       R13
Electricity         Power                                  kW   1.5e+04
                    Cost                               USD/hr  1.18e+03
High pressure steam Duty                                kJ/hr  4.44e+09
                    Flow                              kmol/hr  1.38e+05
                    Cost                               USD/hr  4.38e+04
Design              Diameter                               ft      32.7
                    Length                                 ft       369
                    Reactor volume                         m3  8.79e+03
                    Total volume                           m3  7.03e+04
                    Residence time                         hr         2
                    Turnaround time                        hr         1
                    Batch time                             hr         3
                    Vessel type                                Vertical
                    Weight                                 lb  2.63e+07
                    Wall thickness                         in      15.3
                    Pressure drop                         bar    0.0987
Purchase cost       Pump 1 - Pump                         USD  6.61e+06
                    Pump 1 - Motor (x41)                  USD  4.58e+03
                    Heat exchanger 1 - Floating head      USD  4.89e+06
Total purchase cost                                       USD  1.15e+07
Utility cost                                           USD/hr   4.5e+04

In [53]:
carbohydrate_derivates  = ((solvolysis_reactor.outs[1].imass['Acetate']
  + solvolysis_reactor.outs[1].imass['Cellulose'] + solvolysis_reactor.outs[1].imass['Xylan'])*24*330)/1e6

print(f'Carbohydrate derivates in solvent due to dissolution constitute {carbohydrate_derivates} MMkg/yr \n In Bartling et al, it is 54.2 MM kg/yr')

Carbohydrate derivates in solvent due to dissolution constitute 53.724 MMkg/yr 
 In Bartling et al, it is 54.2 MM kg/yr


In [54]:
hydrogen_in = bst.Stream('Hydrogen_In', 
                         Hydrogen = meoh_in.F_vol*100,   # Anderson et al, had MeOH/H2 feeds as 1:100 vol:vol ratio
                         units = 'm3/hr',
                         T = 190+273.15,
                         P = 6e6,
                         phase = 'g'
)




In [55]:
hydrogenolysis = bst.ParallelReaction([
    bst.Reaction('SolubleLignin,l ->Propylguaiacol,l', reactant = 'SolubleLignin', phases = 'slg', X = 0.25, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l ->Propylsyringol,l', reactant = 'SolubleLignin', phases = 'slg', X = 0.25, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> Syringaresinol,l', reactant= 'SolubleLignin',phases = 'slg', X = 0.125, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> G_Dimer,l', reactant= 'SolubleLignin', phases = 'slg',X = 0.125, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> S_Oligomer,l', reactant= 'SolubleLignin',phases = 'slg', X = 0.125, basis = 'wt', correct_atomic_balance=False),
    bst.Reaction('SolubleLignin,l -> G_Oligomer,l', reactant= 'SolubleLignin', phases = 'slg',X = 0.125, basis = 'wt', correct_atomic_balance=False),
])

In [56]:
import biosteam as bst, numpy as np

from typing import Optional
from biosteam.units.design_tools import (
    PressureVessel, 
)
 


__all__ = (
    'HydrogenolysisReactor', 'RCF2'
    )

class HydrogenolysisReactor(bst.Unit, bst.units.design_tools.PressureVessel):


    auxiliary_unit_names = (
        'heat_exchanger_2'
    )

    _N_ins = 2
    _N_outs = 1
    
    _units = {**PressureVessel._units,
              # 'Pressure drop': 'bar',
              'Diameter': 'm',
              'Length': 'm',
              'Residence time': 'hr',
              'Reactor volume': 'm3'}
    


    # Default operating temperature [K]
    T_default: float = 463.15  # 190 C from  https://doi.org/10.1016/j.joule.2017.10.004

    #: Default operating pressure [Pa]
    P_default:  float = 6e6 # 6 MPa from https://doi.org/10.1016/j.joule.2017.10.004
    
    #: Default residence time [hr]
    tau_default: float = 3

    # Default superficial velocity of solvent (m/s)
    superficial_velocity_default: float = 1 # Just assumed 

    # Default catalyst bed void fraction (epsilon)
    void_frac_default: float = 0.3    


    # Default working volume fraction 
    working_vol_default: float = 0.8
 
    # Default catayyst diameter [m]
    # poplar_diameter_default: float = 0.004   
    



    def _init(
            self,
            T: Optional[float] = None, 
            P: Optional[float] = None,
            tau: Optional[float] = None,
            vessel_material: Optional[str] = None,
            vessel_type: Optional[str] = None,
            superficial_velocity: Optional[float] = None,
            void_frac: Optional[float] = None,
            working_vol: Optional[float] = None,
            *,
            reaction
            ):
        
        
        self.T = self.T_default if T is None else T
        self.P = self.P_default if P is None else P
        self.tau = self.tau_default if tau is None else tau
        self.vessel_material = 'Stainless steel 316' if vessel_material is None else vessel_material
        self.vessel_type = 'Vertical' if vessel_type is None else vessel_type
        self.superficial_velocity = self.superficial_velocity_default if superficial_velocity is None else superficial_velocity
        self.void_frac = self.void_frac_default if void_frac is None else void_frac
        self.working_vol = self.working_vol_default if working_vol is None else working_vol
        self.reaction = reaction
        #pump_1 = self.auxiliary('pump_1', bst.Pump, ins = self.ins[1])
        heat_exchanger_2 = self.auxiliary('heat_exchanger_2', bst.HXutility)


    def _size_bed(self):

        #### Reactor volume sizing ########

        residence_time = self.tau
        occupied_frac = 1 - self.void_frac  
        working_vol = self.working_vol        
        influent = self.ins[0]
        liquid_V_w = influent.F_vol * residence_time # [m3] Total liquid working volume
        V_actual = liquid_V_w/(occupied_frac * working_vol)  # [m3]

        ##### Reactor diameter and length ########
        u  = self.superficial_velocity

        Q_per_reactor_m3 = influent.F_vol    # [m3/hr] Volumetric flow rate processed by any reactor
        self.area = A  = Q_per_reactor_m3/u
        self.diameter = diameter = 2 * (A/np.pi) ** 0.5
        self.length = length = V_actual/A

        
        return length, diameter, V_actual
        
    def _run(self):
        solvent, hydrogen = self.ins
        effluent, = self.outs

        

        effluent.copy_like(solvent)
        self.reaction(effluent)

        h2 = hydrogen.imass['Hydrogen']

        effluent.imass['g', 'Hydrogen'] = h2

        effluent.T = self.T # Assuming isothermal operation
        effluent.P = self.P # Assuming no P drop




    #def _calculate_pressure_drop(self):
        # NOT OPERATIONAL FOR HYDROGENOLYSIS REACTOR

        #D = self.poplar_diameter # [m] poplar particle diameter
        #rho_solv = self.ins[1].rho  # [kg/m3] methanol water density
        #mu = self.ins[1].get_property('mu', 'kg/m/s') # [Pa s] methanol water viscosity 
        #epsilon = self.void_frac # Void fraction 
        #u = self.superficial_velocity # [m/s] superficial velocity          
        
        #Re = (D*rho_solv*u)/mu
        #if Re/(1-epsilon) < 500: # Erun equation
        #    dP = ((1-epsilon)/(epsilon**3))*(1.75+(150*(1-epsilon)/Re))
        #elif 1000 < Re/(1-epsilon) < 5000: # Handley and Heggs
        #    dP = ((1-epsilon)/(epsilon**3))*(1.24+(368*(1-epsilon)/Re))
        #else: # Hicks equation which fits in Wentz and Thodos results for very high Re
        #    dP = 6.8*(((1-epsilon)**1.2)/epsilon**3)*Re**-0.2
        #return dP

        

    def _design(self):
        diameter, length, V_actual = self._size_bed()   # Calling size bed function to determine diameter and length 


        self.set_design_result('Diameter', 'm', diameter)  
        self.set_design_result('Length', 'm', length)
        self.set_design_result('Reactor volume', 'm3', V_actual)
        self.set_design_result('Residence time', 'hr', self.tau)


        
        # Calculates weight based off pressure, diameter and length
        # Adds vcessel type wall thickness, vessel weight, diameter and length to dictionary
        # But diameter and length are already there because of set_design_result above
        self.design_results.update(
            self._vertical_vessel_design(    
                self.P*(1/6894.76),
                self.design_results['Diameter']*3.28084,
                self.design_results['Length']*3.28084
            )
        )
        
                            

        #pressure_drop = self._calculate_pressure_drop()                  
        
        #self.set_design_result('Pressure drop', 'bar', pressure_drop)
        #self.pump_1.P = (self.P - self.ins[1].P) + (pressure_drop*1e5)
        #self.pump_1.simulate()

        #self.heat_exchanger_2.T = self.T
        #self.heat_exchanger_2.simulate()
        





        

    def _cost(self):
        design = self.design_results # Calling the dictionary used to store design results in design method above 

        baseline_purchase_costs = self.baseline_purchase_costs # Dictionary for storing baseline costs

        weight = design['Weight']  # weight parameter stores the value from the 'Weight' key in the design dictionnary

        # Calculates the baseline purchase cost based off diameter length and weight
        baseline_purchase_costs.update( 
            self._vessel_purchase_cost(weight, design['Diameter'], design['Length'])
        )



        
        """



        """



        


In [57]:
hydrogenolysis_reactor = HydrogenolysisReactor(ins = (solvolysis_reactor.outs[1], hydrogen_in), reaction = hydrogenolysis)
hydrogenolysis_reactor.simulate()

C:\Users\hwadg\AppData\Local\Temp\ipykernel_11512\3052927998.py:162: CostWarning: <HydrogenolysisReactor: R2> Vertical vessel weight (7.365e+06 lb) is out of bounds (4200 to 1e+06 lb) for cost correlation
  self._vertical_vessel_design(
C:\Users\hwadg\AppData\Local\Temp\ipykernel_11512\3052927998.py:162: CostWarning: <HydrogenolysisReactor: R2> Vertical vessel length (370 ft) is out of bounds (12 to 40 ft) for cost correlation
  self._vertical_vessel_design(
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Vertical pressure vessel', has no defined bare-module factor in the 'HydrogenolysisReactor.F_BM' dictionary; bare-module factor now has a default value of 1
  warn(f"the purchase cost item, '{name}', has "
c:\Users\hwadg\anaconda3\envs\pyfuel\lib\site-packages\biosteam\_unit.py:986: RuntimeWarning: the purchase cost item, 'Platform and ladders', has no defined bare-module factor in the 'HydrogenolysisReactor.F_BM' 

In [58]:
hydrogenolysis_reactor.results()

Hydrogenolysis reactor                         Units        R2
Design              Diameter                       m      17.6
                    Length                         m       370
                    Reactor volume                m3  5.35e+04
                    Residence time                hr         3
                    Vessel type                       Vertical
                    Weight                        lb  7.36e+06
                    Wall thickness                in      8.27
Purchase cost       Vertical pressure vessel     USD  1.41e+07
                    Platform and ladders         USD  2.13e+05
Total purchase cost                              USD  1.44e+07
Utility cost                                  USD/hr         0

In [59]:
hydrogenolysis_reactor

HydrogenolysisReactor: R2
ins...
[0] s6  from  SolvolysisReactor-R1
    phases: ('g', 'l', 's'), T: 473.15 K, P: 6e+06 Pa
    flow (kmol/hr): (l) Water          4.15e+04
                        SolubleLignin  109
                        Methanol       1.65e+05
                    (s) Extract        7.4
                        Acetate        48.6
                        Cellulose      23.8
[1] Hydrogen_In  
    phase: 'g', T: 463.15 K, P: 6e+06 Pa
    flow (kmol/hr): Hydrogen  1.13e+06
outs...
[0] s7  
    phases: ('g', 'l', 's'), T: 463.15 K, P: 6e+06 Pa
    flow (kmol/hr): (g) Hydrogen        1.13e+06
                    (l) Water           4.15e+04
                        Methanol        1.65e+05
                        Propylguaiacol  25
                        Propylsyringol  21.2
                        Syringaresinol  4.97
                        G_Dimer         5.73
                        S_Oligomer      3.31
                        ...             3.84
                    (s) 